In [ ]:
!pip install boto3

# **Getting Started with BedRock**



In [ ]:
import json
import boto3
client=boto3.client('bedrock-runtime',region_name='us-east-1',aws_access_key_id='',aws_secret_access_key='')
inputText=input("Enter the question for Gen AI : ")
body_part=json.dumps({'inputText': inputText,
 'textGenerationConfig': {'maxTokenCount': 8192,
  'stopSequences': [],
  'temperature': 0,
  'topP': 1}})
response = client.invoke_model(
    body=body_part,
    contentType="application/json",
    accept="application/json",
    modelId='amazon.titan-text-express-v1'
)
output_text=json.loads(response['body'].read())['results'][0]['outputText']
output_text[output_text.index('\n')+1:]

# **Text Embedding with BedRock**

In [ ]:
import json
import boto3
client=boto3.client('bedrock-runtime',region_name='us-east-1',aws_access_key_id='',aws_secret_access_key='')
inputText=input("Enter the word/sentence for embedding : ")
body_part=json.dumps({'inputText': inputText})
response = client.invoke_model(
    body=body_part,
    contentType="application/json",
    accept="application/json",
    modelId='amazon.titan-embed-text-v1'
)
json.loads(response['body'].read())['embedding']

# **Cosine Similarity**

In [ ]:
import json
import boto3
import math
client=boto3.client('bedrock-runtime',region_name='us-east-1',aws_access_key_id='',aws_secret_access_key='')

def get_embedding(b):
  body_part=json.dumps({
 'inputText':b
})
  response=client.invoke_model(
  body=body_part,
  contentType="application/json",
  accept="application/json",
  modelId='amazon.titan-embed-text-v1'
  )
  return json.loads(response['body'].read())['embedding']

def cosine_similarity(v1,v2):
    "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/(math.sqrt(sumxx)*math.sqrt(sumyy))

text1 = "This bottle is empty"
text2 = "There is nothing in the bottle"
v1=get_embedding(text1)
v2=get_embedding(text2)

print(v1)
print(v2)

print(cosine_similarity(v1,v2))


# **GenAI application development with Pinecone Vector Database**

In [ ]:
!pip install pinecone-client boto3

In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key="")
index = pc.Index("demoytsimilaritysearch")

In [ ]:
data_to_store=["Mango","JackFruit","Banana","Red","Green","Blue" ,"Pen","Pensil","Scale"]

import json
import boto3
import math
client=boto3.client('bedrock-runtime',region_name='us-east-1',aws_access_key_id='',aws_secret_access_key='')

def get_embedding(b):
  body_part=json.dumps({
 'inputText':b
})
  response=client.invoke_model(
  body=body_part,
  contentType="application/json",
  accept="application/json",
  modelId='amazon.titan-embed-text-v1'
  )
  return json.loads(response['body'].read())['embedding']


vectors=[]
for i in data_to_store:
  vectors.append({
            "id":i,
            "values": get_embedding(i)
        })

index.upsert(
    vectors=vectors,
    namespace= "ns1"
)

In [ ]:
index.query(
    namespace="ns1",
    vector= get_embedding("Ruler") ,
    top_k=2
)

# **Multimodal Embeddings with BedRock**

# [**Image Dataset**](https://github.com/SatadruMukherjee/Data-Preprocessing-Models/blob/main/Multimodal%20Embedding%20using%20BedRock.zip)

In [ ]:
!pip install boto3 pinecone-client

In [ ]:
#Reference Document: https://aws.amazon.com/blogs/aws/amazon-titan-image-generator-multimodal-embeddings-and-text-models-are-now-available-in-amazon-bedrock/
import json
import boto3
import base64

bedrock_runtime=boto3.client('bedrock-runtime',region_name='us-east-1',aws_access_key_id='',aws_secret_access_key='')

from pinecone import Pinecone

pc = Pinecone(api_key="{}")
index = pc.Index("imagesearchytdemo")


def image_embedding(image_name):
  # Maximum image size supported is 2048 x 2048 pixels
  with open(image_name, "rb") as image_file:
    input_image = base64.b64encode(image_file.read()).decode('utf8')

  # You can specify either text or image or both
  body = json.dumps(
    {
        "inputImage": input_image
    }
  )
  response = bedrock_runtime.invoke_model(
  body=body,
  modelId="amazon.titan-embed-image-v1",
  accept="application/json",
  contentType="application/json"
  )
  response_body = json.loads(response.get("body").read())
  return response_body.get("embedding")

data_to_store=["1.png", "2.png", "3.png", "4.png", "5.png", "6.png", "7.png"]

vectors=[]
for i in data_to_store:
  vectors.append({
            "id":i,
            "values": image_embedding(i)
        })

index.upsert(
    vectors=vectors,
    namespace= "ns1"
)

In [ ]:
index.query(
    namespace="ns1",
    vector=image_embedding('/content/red_bag_testing.PNG'),
    top_k=2
)

In [ ]:
def text_multimodal_embedding(text):

  # You can specify either text or image or both
  body = json.dumps(
    {
        "inputText": text
    }
  )
  response = bedrock_runtime.invoke_model(
  body=body,
  modelId="amazon.titan-embed-image-v1",
  accept="application/json",
  contentType="application/json"
  )
  response_body = json.loads(response.get("body").read())
  return response_body.get("embedding")

index.query(
    namespace="ns1",
    vector=text_multimodal_embedding('Pink Bag'),
    top_k=2
)

# **RAG**

In [ ]:
!pip install boto3 langchain langchain-pinecone

Reference Links:


https://github.com/aws-samples/amazon-bedrock-samples/blob/main/rag-solutions/rag-foundations-workshop/notebooks/03_retrieval_based_text_application.ipynb

https://python.langchain.com/docs/integrations/vectorstores/pinecone/

In [ ]:
import boto3
import os


boto3_bedrock = boto3.client('bedrock-runtime',region_name='us-east-1',aws_access_key_id='',aws_secret_access_key='')


In [ ]:
from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader
from langchain.embeddings import BedrockEmbeddings
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
# load the sagemaker FAQ list
with open('/content/yt_ran_dem.txt') as f:
    doc = f.read()

# create a loader
docs = []
loader = TextLoader('')
docs.append(Document(page_content=doc))

# split documents into chunks
text_splitter = CharacterTextSplitter(
    separator='\n',
    chunk_size=300,
    chunk_overlap=25,

)
split_docs = text_splitter.split_documents(docs)

In [ ]:
split_docs

In [ ]:
import os
os.environ['PINECONE_API_KEY']='{}'

In [ ]:
embedding_model = BedrockEmbeddings(
    client=boto3_bedrock,
    model_id="amazon.titan-embed-text-v1"
)
from langchain_pinecone import PineconeVectorStore

index_name = "demoytrag"

docsearch = PineconeVectorStore.from_documents(split_docs, embedding_model, index_name=index_name)

In [ ]:
search_results = docsearch.similarity_search(
    'What is the bank name mentioned in the blog?', k=3
)
print(search_results)

In [ ]:
from langchain import PromptTemplate

RAG_PROMPT_TEMPLATE = '''Here is some important context which can help inform the questions the Human asks.
Make sure to not make anything up to answer the question if it is not provided in the context.

<context>
{context}
</context>

Human: {human_input}

Assistant:
'''
PROMPT = PromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

In [ ]:
import json

In [ ]:
human_input = input("Enter your question : ")
search_results = docsearch.similarity_search(human_input, k=3)
context_string = '\n\n'.join([f'Document {ind+1}: ' + i.page_content for ind, i in enumerate(search_results)])
prompt_data = PROMPT.format(human_input=human_input, context=context_string)
inputText=prompt_data
body_part=json.dumps({'inputText': inputText,
 'textGenerationConfig': {'maxTokenCount': 8192,
  'stopSequences': [],
  'temperature': 0,
  'topP': 1}})
response = boto3_bedrock.invoke_model(
    body=body_part,
    contentType="application/json",
    accept="application/json",
    modelId='amazon.titan-text-express-v1'
)
output_text=json.loads(response['body'].read())['results'][0]['outputText']
output_text[output_text.index('\n')+1:]